In [ ]:
!pip install langchain-community langchain-core

In [ ]:
!pip install langchain-huggingface
!pip install langchain-experimental

In [ ]:
!pip install json-repair
!pip install langchain-huggingface transformers accelerate
!pip install langchain neo4j

In [ ]:
import os
from langchain_huggingface import ChatHuggingFace
from langchain_core.messages import HumanMessage
from langchain_community.llms import HuggingFaceEndpoint # Required for ChatHuggingFace
from langchain_huggingface import HuggingFacePipeline
from langchain_experimental.graph_transformers import LLMGraphTransformer
import torch
from transformers import pipeline
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_huggingface.chat_models import ChatHuggingFace
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "*****"
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

hf_llm_pipeline = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from neo4j import GraphDatabase

URI = "*****"
USERNAME = "neo4j"
PASSWORD = "*****"

In [ ]:

# Connecting to the Neo4j graph using the LangChain Neo4jGraph class
graph_db = Neo4jGraph(url=URI, username=USERNAME, password=PASSWORD)

In [ ]:
from langchain_community.graphs import Neo4jGraph

# graph_db = Neo4jGraph()

q_load_articles = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/dcarpintero/generative-ai-101/main/dataset/synthetic_articles.csv'
AS row
FIELDTERMINATOR ';'
MERGE (a:Article {title:row.Title})
SET a.abstract = row.Abstract,
    a.publication_date = date(row.Publication_Date)
FOREACH (researcher in split(row.Authors, ',') |
    MERGE (p:Researcher {name:trim(researcher)})
    MERGE (p)-[:PUBLISHED]->(a))
FOREACH (topic in [row.Topic] |
    MERGE (t:Topic {name:trim(topic)})
    MERGE (a)-[:IN_TOPIC]->(t))
"""

graph_db.query(q_load_articles)

In [ ]:
graph_db.refresh_schema()
print(graph_db.get_schema)

In [ ]:
# Load secrets/credentials from local dotenv file
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
!pip install langchain-cohere

In [ ]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-v3.0"
)

In [ ]:
from langchain_community.vectorstores import Neo4jVector
# from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url=os.environ['NEO4J_URI'],
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
    index_name='articles',
    node_label="Article",
    text_node_properties=['topic', 'title', 'abstract'],
    embedding_node_property='embedding',
)

In [ ]:
from langchain_cohere import ChatCohere

# Initialize the ChatCohere model
# Make sure you have the COHERE_API_KEY environment variable set
cohere_llm = ChatCohere(model="command-r-plus-08-2024") # You can specify a different Cohere model here

In [ ]:
from langchain.chains import RetrievalQA

vector_qa = RetrievalQA.from_chain_type(
    llm=cohere_llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)

In [ ]:
r = vector_qa.invoke(
    {"query": "which articles discuss how AI might affect our daily life? include the article titles and abstracts."}
)
print(r['result'])

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "*****"
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

hf_llm_pipeline = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# Connecting to the Neo4j graph using the LangChain Neo4jGraph class
# graph_db = Neo4jGraph(url=URI, username=USERNAME, password=PASSWORD)


CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template="""Given the following Neo4j schema:
{schema}
Generate a Cypher query to answer the question: {question}
""",
)


# Initialize the GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=hf_llm_pipeline, # Pass the initialized HuggingFacePipeline
    graph=graph_db,
    verbose=True,
    allow_dangerous_requests=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)

In [ ]:
# MATCH (r:Researcher {name: "Emily Chen"})-[:PUBLISHED]->(a:Article)
# RETURN COUNT(a) AS numberOfArticles

# the answer should be '7'
chain.invoke(
    {"query": "How many articles has published Emily Chen?"}
)

In [ ]:
# The correct Cypher query to find pairs of researchers who have published more than three articles together is:
# MATCH (r1:Researcher)-[:PUBLISHED]->(article:Article)<-[:PUBLISHED]-(r2:Researcher)
# WHERE r1 <> r2
# WITH r1, r2, count(article) AS shared_articles
# WHERE shared_articles > 3
# RETURN r1.name AS researcher1, r2.name AS researcher2, shared_articles

result = chain.invoke(
    {"query": "are there any pair of researchers who have published more than three articles together?"}
)
print(result['result'])

In [ ]:
# MATCH (r:Researcher)-[:PUBLISHED]->(:Article)<-[:PUBLISHED]-(peer:Researcher)
# WITH r, COUNT(DISTINCT peer) AS peerCount
# RETURN r.name AS researcher, peerCount
# ORDER BY peerCount DESC
# LIMIT 1

# the answer should be 'David Johnson'
result = chain.invoke(
    {"query": "Which researcher has collaborated with the most peers?"}
)
print(result['result'])